<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo\">

# `dask.delayed`并行编程

在本节中，我们用Dask和`dask.delayed`并行处理简单的for-loop风格的代码。通常情况下，这是您需要转换用于Dask的函数的唯一函数。

这是使用`dask`来并行化现有代码库或构建[复杂系统](https://blog.dask.org/2018/02/09/credit-models-with-dask)的简单方法。 这也将帮助我们对后面的章节形成理解。

**相关文档**

* [Delayed documentation](https://docs.dask.org/en/latest/delayed.html)
* [Delayed screencast](https://www.youtube.com/watch?v=SHqFmynRxVU)
* [Delayed API](https://docs.dask.org/en/latest/delayed-api.html)
* [Delayed examples](https://examples.dask.org/delayed.html)
* [Delayed best practices](https://docs.dask.org/en/latest/delayed-best-practices.html)

正如我们在[分布式调度器笔记本](05_distributed.ipynb)中看到的，Dask有几种并行执行代码的方式。我们将通过创建一个`dask.distributed.Client`来使用分布式调度器。目前，这将为我们提供一些不错的诊断。我们将在后面深入讨论调度器。

In [ ]:
from dask.distributed import Client

client = Client(n_workers=4)

## 基础

首先让我们做一些玩具函数，`inc`和`add`，它们会睡一会儿来模拟工作。然后我们会定时正常运行这些函数。

在下一节中，我们将对这段代码进行并行处理。

In [ ]:
from time import sleep

def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y

我们使用"%%time "魔法对这段普通代码的执行进行计时，这是Jupyter Notebook的一个特殊函数。

In [ ]:
%%time
# This takes three seconds to run because we call each
# function sequentially, one after the other

x = inc(1)
y = inc(2)
z = add(x, y)

### 使用`dask.delayed`装饰器并行化

这两个增量调用*可以并行调用，因为它们完全相互独立。

我们将使用`dask.delayed`函数来转换`inc`和`add`函数。当我们通过传递参数来调用延迟版本时，与之前完全一样，原始函数实际上还没有被调用--这就是为什么单元格的执行很快就结束了。
取而代之的是一个*delayed对象*，它跟踪要调用的函数和要传递给它的参数。


In [ ]:
from dask import delayed

In [ ]:
%%time
# 这将立即运行，它所做的就是建立一个图

x = delayed(inc)(1)
y = delayed(inc)(2)
z = delayed(add)(x, y)

这个立即运行，因为还没有真正发生什么。

为了得到结果，调用`compute`。请注意，这比原来的代码运行得更快。

In [ ]:
%%time
# 这实际上是使用一个本地线程池来运行我们的计算。

z.compute()

## 刚才发生了什么？

`z`对象是一个懒惰的`Delayed`对象。 这个对象包含了我们计算最终结果所需要的一切，包括对所有需要的函数的引用以及它们的输入和彼此之间的关系。 我们可以像上面一样用`.compute()`来评估结果，也可以用`.visualize()`来可视化这个值的任务图。

In [ ]:
z

In [ ]:
# 看`z`的任务图。
z.visualize()

请注意，这包括之前的函数名称，以及 "inc "函数的输出到 "add "输入的逻辑流程。

### 一些需要考虑的问题

- 为什么我们要从3秒变成2秒？ 为什么我们不能并行化到1s？
- 如果 inc 和 add 函数不包含 `sleep(1)`，会发生什么？ Dask 还能加快这段代码的速度吗？
- 如果我们有多个输出，或者也想访问x或y，怎么办？

## 练习: 并行化一个for循环

`for`循环是我们想要并行化的最常见的事情之一。 在`inc`和`sum`上使用`dask.delayed`来并行化下面的计算。

In [ ]:
data = [1, 2, 3, 4, 5, 6, 7, 8]

In [ ]:
%%time
# Sequential code

results = []
for x in data:
    y = inc(x)
    results.append(y)
    
total = sum(results)

In [ ]:
total

In [ ]:
%%time
# Your parallel code here...

In [ ]:
results = []

for x in data:
    y = delayed(inc)(x)
    results.append(y)
    
total = delayed(sum)(results)
print("Before computing:", total)  # Let's see what type of thing total is
result = total.compute()
print("After computing :", result)  # After it's computed

与给定的方案相比，与直接使用`sum`函数而不是用`delayed`包装的版本相比，图形可视化效果如何？你能解释一下后一个版本吗？你可能会发现以下表达式的结果很有启发性。
```python
delayed(inc)(1) + delayed(inc)(2)
```

## 练习。并行化一个带有控制流的for-loop代码。

通常我们只想延迟*部分函数，立即运行其中的几个函数。 当这些函数速度很快的时候，这特别有用，可以帮助我们确定应该调用哪些其他较慢的函数。 在使用`dask.delayed`的时候，这个延迟或不延迟的决定，通常是我们需要深思熟虑的地方。

在下面的例子中，我们迭代一个输入列表。 如果该输入是偶数，那么我们要调用`inc`。 如果该输入是奇数，那么我们要调用`double`。 为了使我们的Python代码能够继续进行，必须立即做出调用inc`或double`的`is_even`决定(而不是懒惰地)。

In [ ]:
def double(x):
    sleep(1)
    return 2 * x

def is_even(x):
    return not x % 2

data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
%%time
# Sequential code

results = []
for x in data:
    if is_even(x):
        y = double(x)
    else:
        y = inc(x)
    results.append(y)
    
total = sum(results)
print(total)

In [ ]:
%%time
# Your parallel code here...
# TODO: 使用dask.delayed并行处理上面的顺序执行代码
# 你需要延迟一些函数，但不是所有的函数

In [ ]:
results = []
for x in data:
    if is_even(x):  # even
        y = delayed(double)(x)
    else:          # odd
        y = delayed(inc)(x)
    results.append(y)
    
total = delayed(sum)(results)

In [ ]:
%time total.compute()

In [ ]:
total.visualize()

### 一些问题需要考虑:

- 还有哪些控制流的例子是我们不能使用延迟的？
- 在上面的例子中，如果我们延迟了`is_even(x)`的评估，会发生什么？
- 对于延迟`sum`，你有什么想法？ 这个函数既能计算，又能快速运行。

## 练习: 通过还原并行化潘达斯集团

在这个练习中，我们读取几个CSV文件，并并行执行一个groupby操作。 我们得到了做这个操作的顺序代码，并用`dask.delayed`进行并行化。

我们要并行的计算是根据一些历史航班数据计算每个机场的平均起飞延迟。 我们将通过使用`dask.delayed`和`pandas`来完成。 在未来的一节中，我们将用`dask.dataframe`做同样的练习。

## 创建数据

运行这段代码来准备一些数据。

这将下载并提取1990年至2000年期间纽约市外的一些历史航班数据。这些数据最初来自[这里](http://stat-computing.org/dataexpo/2009/the-data.html)。

In [ ]:
%run prep.py -d flights

### Inspect data

In [ ]:
import os
sorted(os.listdir(os.path.join('data', 'nycflights')))

### 用`pandas.read_csv`读取一个文件，并计算平均出发延迟。

In [ ]:
import pandas as pd
df = pd.read_csv(os.path.join('data', 'nycflights', '1990.csv'))
df.head()

In [ ]:
# 模式是什么？
df.dtypes

In [ ]:
# 数据中有哪些始发机场？
df.Origin.unique()

In [ ]:
# 每个机场一年的平均离港延误时间
df.groupby('Origin').DepDelay.mean()

### 顺序代码：每个机场的平均起飞延误时间

上面的单元格计算了一年中每个机场的平均起飞延误。这里，我们使用一个连续的for循环将其扩展到所有年份。

In [ ]:
from glob import glob
filenames = sorted(glob(os.path.join('data', 'nycflights', '*.csv')))

In [ ]:
%%time

sums = []
counts = []
for fn in filenames:
    # Read in file
    df = pd.read_csv(fn)
    
    # Groupby origin airport
    by_origin = df.groupby('Origin')
    
    # Sum of all departure delays by origin
    total = by_origin.DepDelay.sum()
    
    # Number of flights by origin
    count = by_origin.DepDelay.count()
    
    # Save the intermediates
    sums.append(total)
    counts.append(count)

# Combine intermediates to get total mean-delay-per-origin
total_delays = sum(sums)
n_flights = sum(counts)
mean = total_delays / n_flights

In [ ]:
mean

### 将上面的代码并行化

使用`dask.delayed`来并行化上面的代码。 一些额外的事情你需要知道。

1.  延迟对象上的方法和属性访问会自动工作，所以如果你有一个延迟对象，你可以对它进行正常的运算、切片和方法调用，它会产生正确的延迟调用。

    ```python
    x = delayed(np.arange)(10)
    y = (x + 1)[::2].sum()  # everything here was delayed
    ```
2.  当你有单一输出时，调用`.compute()`方法效果很好。 当你有多个输出时，你可能想使用`dask.compute`函数。

    ```python
    >>> from dask import compute
    >>> x = delayed(np.arange)(10)
    >>> y = x ** 2
    >>> min_, max_ = compute(y.min(), y.max())
    >>> min_, max_
    (0, 81)
    ```
    
    这样Dask就可以共享中间值（如`y = x**2`）。

所以你的目标是使用`dask.delayed`来并行上面的代码（已经复制到下面）。 你可能还想可视化一下计算，看看你做的是否正确。

In [ ]:
from dask import compute

In [ ]:
%%time

# copied sequential code

sums = []
counts = []
for fn in filenames:
    # Read in file
    df = pd.read_csv(fn)
    
    # Groupby origin airport
    by_origin = df.groupby('Origin')
    
    # Sum of all departure delays by origin
    total = by_origin.DepDelay.sum()
    
    # Number of flights by origin
    count = by_origin.DepDelay.count()
    
    # Save the intermediates
    sums.append(total)
    counts.append(count)

# 合并中间值，得到总的平均每原点延迟时间
total_delays = sum(sums)
n_flights = sum(counts)
mean = total_delays / n_flights

In [ ]:
mean

In [ ]:
%%time
# your code here

如果你加载了解决方案，在单元格顶部添加`%%time`来测量运行时间。

In [ ]:
%%time

# 这只是一个可能的解决方案，有几种方法可以使用 "延迟 "来实现。

sums = []
counts = []
for fn in filenames:
    # Read in file
    df = delayed(pd.read_csv)(fn)

    # Groupby origin airport
    by_origin = df.groupby('Origin')

    # Sum of all departure delays by origin
    total = by_origin.DepDelay.sum()

    # Number of flights by origin
    count = by_origin.DepDelay.count()

    # Save the intermediates
    sums.append(total)
    counts.append(count)

# Compute the intermediates
sums, counts = compute(sums, counts)

# Combine intermediates to get total mean-delay-per-origin
total_delays = sum(sums)
n_flights = sum(counts)
mean = total_delays / n_flights

In [ ]:
# ensure the results still match
mean

### 一些问题需要考虑：

- 你的速度提升了多少？这是你期望的速度吗？
- 实验一下在哪里调用 "compute"。当你在 "sums "和 "counts "上调用它会发生什么？如果你等待并在 "mean "上调用它会发生什么？
- 实验一下延迟调用`sum`。如果延迟调用`sum`，图形是什么样子的？如果不延时，图形是什么样子的？
- 你能想到为什么要用一种方式而不是另一种方式进行还原吗？

### 了解更多

访问[delayed文档](https://docs.dask.org/en/latest/delayed.html)。特别是这个[延迟屏幕广播](https://www.youtube.com/watch?v=SHqFmynRxVU)将强化你在这里学到的概念，[延迟最佳实践](https://docs.dask.org/en/latest/delayed-best-practices.html)文档收集了关于如何使用好`dask.delayed`的建议。

## 关闭客户端

在进行下一个练习之前，请确保关闭你的客户端或停止这个内核。

In [ ]:
client.close()